In [ ]:
import json
import re
from bson import ObjectId  # Simulating MongoDB's ObjectId

def hash_password(password):
    return "hashed_" + password  # Simulated hashing function

def parse_sql_to_json(sql, user_id):
    pattern = r"CALL insert_user\((.*?)\);"
    match = re.search(pattern, sql)
    
    if not match:
        raise ValueError("Invalid SQL format")
    
    values = [v.strip().strip("'") if v.strip() != "NULL" else None for v in match.group(1).split(",")]
    
    device_type, officer_id, name, password, phone, email, reg_date, birth_date, user_type = values
    
    json_data = {
        "_id": str(ObjectId()),
        "user_type": user_type.replace("registerred", "registered").replace("unregisterred", "unregistered"),
        "device_type": device_type,
    }
    
    if user_type == "registerred_user" or user_type == "admin":
        json_data["user_details"] = {
            "name": name,
            "email": email,
            "password": hash_password(password) if password else None,
            "telephone_number": phone,
            "register_date": reg_date,
            "birth_date": birth_date
        }
    
    if user_type == "admin":
        json_data["admin_info"] = {"officerID": officer_id}
    
    return json.dumps(json_data, indent=2)

# Example usage
sql1 = "CALL insert_user('Android', NULL, 'Kwanjai Chuchai', 'regpass456', '1167890123', 'kwanjai.chuchai@example.com', CAST(NOW() AS TIMESTAMP), '1993-01-19', 'registerred_user');"
sql2 = "CALL insert_user('iOS', NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'unregisterred_user');"
sql3 = "CALL insert_user('iOS', 113, 'Sittipong Wongsakul', 'password123', '1090123456', 'sittipong.wongsakul@example.com', CAST(NOW() AS TIMESTAMP), '1987-09-11', 'admin');"

print(parse_sql_to_json(sql1, "user_id_1"))
print(parse_sql_to_json(sql2, "user_id_2"))
print(parse_sql_to_json(sql3, "user_id_3"))